In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/",one_hot = True)

C:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [2]:
learning_rate = 0.001
trining_epoch = 15
batch_size = 100

In [3]:
X = tf.placeholder(tf.float32,shape = [None, 784])
Y = tf.placeholder(tf.float32,shape = [None, 10])


# convolutinoal을 사용하고 싶으면 x_image로 reshpae을 하는 단계를 꼭 거쳐야한다.
X_img = tf.reshape(X, [-1,28,28,1]) # 여기서 -1 은 None과 같다.

#필터갯수를 정해주는 것이다 (W1은) 
# 필터의 weight를 초기화할때 sigma값을 작게줘서 그 범위를 줄여서 랜덤값의 서로 다른 정도를 줄인다. 
W1 = tf.Variable(tf.random_normal([3,3,1,32],stddev = 0.01))
L1 = tf.nn.conv2d(X_img, W1, strides = [1,1,1,1],padding = "SAME")
# 결과값 L1 image --> -1 * 28 * 28 * 32
# -1 = None = n개

# Relu층
L1 = tf.nn.relu(L1)

# Pooling 층
L1 = tf.nn.max_pool(L1,ksize = [1,2,2,1], strides = [1,2,2,1], padding= "SAME") # strides가 1을 기준으로 ZEROPADDING하고 POOLING한다.
# 절반으로 줄이는 방법 - strides = [1,2,2,1], padding= "SAME"
# 나머지가 없으면 zeropadding에서 추가하지 않는다
# 시계방향해서 가로 , 세로

# L1 = -1 * 14 * 14 * 32

W2 =  tf.Variable(tf.random_normal([3,3,32,64],stddev = 0.01))
# [3,3,32,64]에서 32는 앞의 과정에서 ouput depth이자 input depth이므로 depth가 같아야한다. 
L2 = tf.nn.conv2d(L1, W2, strides = [1,1,1,1],padding = "SAME")
# L2 = -1 * 14 * 14 * 64

# Relu층
L2 = tf.nn.relu(L2)

# Pooling층
L2 = tf.nn.max_pool(L2,ksize = [1,2,2,1], strides = [1,2,2,1], padding= "SAME")
# L2 = -1 * 7 * 7 * 64

# full connected를 위해서는 flat(하나의 벡터로 바꿔줘야한다.)
L2_flat = tf.reshape(L2, [-1, 7*7*64])

# 모든 layer를 받고 이를 계산할 수 잇는 weight의 shape은 위의 reshape한거, label의 갯수
W3 = tf.get_variable("W3", shape = [7*7*64,10])
logits = tf.matmul(L2_flat,W3)

hypothesis = tf.nn.softmax(logits)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate)
train = optimizer.minimize(cost)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [4]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
for epoch in range(trining_epoch):
    total_batch = int(mnist.train.num_examples/batch_size)
    avg_cost = 0
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        # slicing을 해서 x데이터와 y데이터를 나눠서 반환해준다.
        c, _ = sess.run([cost,train], feed_dict = {X:batch_xs,Y:batch_ys})
        avg_cost += c/total_batch
        # cost를 다 더해서 total_Batch로 나눈것과 같은 효과
    print("Epoch",epoch, "Cost",avg_cost )

Epoch 0 Cost 0.2972486854005945
Epoch 1 Cost 0.09225254363126362
Epoch 2 Cost 0.07136099373921759
Epoch 3 Cost 0.057548705363138115
Epoch 4 Cost 0.05210557778027243
Epoch 5 Cost 0.04354578142795206
Epoch 6 Cost 0.039578057710419966
Epoch 7 Cost 0.034087156947405835
Epoch 8 Cost 0.030138373713813362
Epoch 9 Cost 0.027183885505020773
Epoch 10 Cost 0.02331598669930826
Epoch 11 Cost 0.02101135426095093
Epoch 12 Cost 0.01850028608925641
Epoch 13 Cost 0.016360931184449735
Epoch 14 Cost 0.014985771152185563


In [7]:
x_test = mnist.test.images
y_test = mnist.test.labels

In [9]:
is_correct = tf.equal(tf.argmax(hypothesis,1),tf.argmax(Y,1))
# hypothesis(예측값) 중 가장 큰 값의 index를 뽑아내는 것
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
sess.run(accuracy, feed_dict={X:x_test,Y:y_test})
# accuracy할때는 1을 넣어야한다.

0.9875